In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append('../')
import pickle

from config import *

configure_pylab()
L  = 128

Available files:
	r2288_180515b_tet2_cell2_GC.mat
	r2289_250515b_tet2_cell2_GC.mat
	r2298_290615c_tet6_cell6_GC.mat
	r2375_070416a_tet1_cell1_GC.mat
	r2375_140416b_tet1_cell1_GC.mat
	r2375_210416a_tet3_cell3_GC.mat
	r2383_160516b_tet4_cell4_GC.mat
	r2405_011216a_cell2955.mat
	r2405_051216b_cell1816.mat
	r2405_121216b_cell1880.mat
	r2405_121216b_cell1912.mat
	r2405_191216c_cell1640.mat
	r2405_191216c_cell1662.mat
	r2405_191216c_cell1962.mat
	r2405_191216c_cell1990.mat
Fitted models in fitdir = /home/mer49/Desktop/lgcpspatial/python/notebooks/hyperparameter_fits/


In [2]:
rp = 1.5  # Range (ratio) to search for optimal period
rv = 10   # Range (ratio) to search for optimal kernel height
Np = 101  # Period grid resolution
Nv = 101  # Kernel height grid resolution
Nθ = 60   # Angles to test

ff = datafiles
shuffle(ff)

for f in datafiles:
    print('FILE:',f)
    fn   = datadir+f
    data = Dataset.from_file(fn).prepare(L)
    saveto = fn.split('/')[-1][:-4]+'.hyperparameters.p'
    if saveto in os.listdir('.'):
        P,V,θ = pickle.load(open(saveto,'rb'))
        continue
        
    # Prepare hyperparameter grid
    Po = data.P # Initial guess for kernel period
    Vo = data.V # Initial guess for kernel height 
    θo = data.angle
    Ps = float32(exp(linspace(log(Po/rp),log(Po*rp),Np)))
    Vs = float32(exp(linspace(log(Vo/rv),log(Vo*rv),Nv))[::-1])
    θs = linspace(0,pi/3,Nθ+1)[:Nθ]
    
    # ~More kosher to use same Fourier components in all models
    kw = dict(P=Po,V=Vo,style='radial')
    kept = full(data.shape,False)
    for Pi in Ps: kept |= kernelft(data.shape,**(kw|dict(P=Pi)))>0
    for Vi in Vs: kept |= kernelft(data.shape,**(kw|dict(V=Vi)))>0
    for θi in θs: kept |= kernelft(data.shape,**(kw|dict(angle=θi)))>0   
    kw |= dict(kept=kept)
    print('total retained components: %d (%d%%)'%(sum(kept),100*mean(kept)))
    
    # Initial inference ignoring orientation
    NKμ = (data.N,data.K,data.prior_mean)
    P,V = grid_search([Ps,Vs],lambda p,s:lgcp2d(
        kernelft(data.shape,*p,kept=kept),*NKμ,initial_guess=s
    )[0]).pars
    print('(P,V)',(P,V))
    
    # Fit orientation
    kw = dict(P=P,V=V,style='grid',angle=θo,kept=kept)
    def angle_helper(θi):
        kernel = kernelft(data.shape,**(kw|dict(angle=θi)))
        return lgcp2d(kernel,*NKμ)[0].ll
    llh = float32([*parmap(angle_helper,θs)])
    θ = θs[argmax(llh)]
    print('θ',θ)
    
    # Re-optimize period and variance
    kw = dict(P=P,V=V,style='grid',angle=θ)
    P,V = grid_search([Ps,Vs],lambda p,s:lgcp2d(
        kernelft(data.shape,*p,style='grid',angle=θ,kept=kept),
        *NKμ,initial_guess=s)[0]).pars
    print('(P,V)',(P,V))
    
    pickle.dump((P,V,θ),open(saveto,'wb'))

FILE: r2405_191216c_cell1990.mat
total retained components: 277 (2%)
[42,41](2.03e+01,2.95e-01) loss=-8.110813e+03(done)
(P,V) (20.092941, 0.29514855)
θ 0.3141592653589793                                                  
[42,39](2.03e+01,3.24e-01) loss=-8.095398e+03(done)
(P,V) (20.256542, 0.3388759)
FILE: r2405_121216b_cell1880.mat
total retained components: 225 (1%)
[35,43](2.11e+01,1.33e-01) loss=-2.011524e+04(done)
(P,V) (21.137562, 0.13968772)
θ 0.47123889803846897                                                 
[39,37](2.18e+01,1.76e-01) loss=-2.009017e+04(done)
(P,V) (21.6581, 0.17585641)
FILE: r2405_191216c_cell1662.mat
total retained components: 315 (2%)
[38,39](1.86e+01,2.63e-01) loss=-6.117420e+03(done)
(P,V) (18.585878, 0.2750376)
θ 0.3490658503988659                                                  
[31,37](1.76e+01,2.88e-01) loss=-6.105820e+03(done)
(P,V) (17.560234, 0.2750376)
FILE: r2405_051216b_cell1816.mat
total retained components: 263 (2%)
[53,53](2.32e+01,2.44e-0